In [1]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.datasets import load_iris, load_breast_cancer, load_wine
from sporboost.forest import *
import pandas as pd
from time import time
import os
from sklearn.metrics import make_scorer
from xgboost import XGBClassifier

################################################################################
################################################################################
################################################################################
# Data Load
def load_simulated(path):
    data = np.loadtxt(path, delimiter=",")
    return data[:,:-1], data[:,-1].astype('int32')

dfs = {
    'iris' : load_iris(return_X_y = True),
    # 'breast_cancer' : load_breast_cancer(return_X_y = True),
    # 'wine' : load_wine(return_X_y = True),
    # 'trunk' : load_simulated("source_data/simulated/Trunk_train.csv"),
    # 'orthant' : load_simulated("source_data/simulated/Orthant_train.csv"),
    # 'sparse_parity' : load_simulated("source_data/simulated/Sparse_parity_train.csv")
}


In [2]:
# clf = AdaBoost(max_depth=2)
# clf.fit(*dfs['iris'])

In [3]:
# clf.predict(dfs['iris'][0])

In [4]:
################################################################################
################################################################################
################################################################################
# Hyperparameters across all estimators
# Boosted Forest
boosted_max_depth = (1, 2, 3)

# Sparse Random Trees
d_ratio = (0.25, 0.5, 0.75, 1)
s_ratio = (1., 2., 3., 4., 5.)

# Rotational Decision Trees
K = (2, 3)


In [5]:
################################################################################
################################################################################
################################################################################
# Scoring Functions
scoring = {
    "cohens_kappa" : make_scorer(cohen_kappa_score),
    "accuracy" : 'accuracy'
}

In [6]:
################################################################################
################################################################################
################################################################################
# Initalize cross-validation params
pipe = Pipeline([
    ('clf', RandomForest())
])

params = [
    {
        'clf' : (RandomForest(),),
    },
    {
        'clf' : (AdaBoost(),),
        'clf__max_depth' : boosted_max_depth
    },
    {
        'clf' : (SPORF(),),
        'clf__d_ratio' : d_ratio,
        'clf__s_ratio' : s_ratio
    },
    {
        'clf' : (SPORBoost(),),
        'clf__max_depth' : boosted_max_depth,
        'clf__d_ratio' : d_ratio,
        'clf__s_ratio' : s_ratio
    },
    {
        'clf' : (RotationalForest(),),
        'clf__K' : K
    },
    {
        'clf' : (RotBoost(),),
        'clf__max_depth' : boosted_max_depth,
        'clf__K' : K
    },
    {
        'clf' : (XGBClassifier(nthread = 1, use_label_encoder=False), )
    }
]

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=20220329)

cv_base = GridSearchCV(estimator = pipe,
                       param_grid = params,
                       cv=kfold,
                       n_jobs=-1,
                       verbose=3,
                       refit=False,
                       scoring=scoring
)


In [12]:
################################################################################
################################################################################
################################################################################
# Func to write results to disk
def write_results(cv, label):
    front_cols = ['param_clf',
                  'mean_test_cohens_kappa', 'std_test_cohens_kappa',
                  'mean_test_accuracy', 'std_test_accuracy',
                  'param_clf__max_depth',
                  'param_clf__d_ratio', 'param_clf__s_ratio',
                  'param_clf__K'
                ]

    results = pd.DataFrame(cv.cv_results_).drop(['params'], axis=1)
    results['param_clf'] = results['param_clf'].apply(lambda c: type(c).__name__)
    results = results.sort_values(['param_clf', 'mean_test_cohens_kappa'],
                                ascending=False, ignore_index = True)
            
    results = results[front_cols + results.drop(front_cols, axis=1).columns.tolist()]

    # All Results
    results.to_csv(f"report/{label}_all.csv", index=False)

    # Best Results
    results.groupby("param_clf").nth(0).reset_index(drop=False).to_csv(f"report/{label}_best.csv", index=False)

In [8]:
################################################################################
################################################################################
################################################################################
# Fit the CVs
for name, df in dfs.items():
    if not os.path.exists(f"report/{name}_all.csv"):
        start = time()
        print(f"Testing {name}")
        res = cv_base.fit(*df)
        write_results(cv_base.fit(*df), name)
        end = time()
        print(f"Finished {name} in {(end - start) / 60:.1f} mins")
    else:
        print(f"Skipping {name} as it has already been tested")

Testing iris
Fitting 10 folds for each of 69 candidates, totalling 690 fits


C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.88 0.94 0.92 0.8  0.8  0.94 0.93 0.95 0.78 0.78 0.94 0.94 0.95 0.77
 0.77 0.95 0.94 0.96 0.8  0.8  0.94 0.97 0.94 0.78 0.78 0.95 0.96 0.95
 0.77 0.77 0.95 0.96 0.94 0.8  0.8  0.93 0.96 0.96 0.78 0.78 0.95 0.95
 0.94 0.77 0.77 0.94 0.94 0.94 0.8  0.8  0.95 0.96 0.94 0.78 0.78 0.95
 0.95 0.95 0.77 0.77 0.95  nan 0.95 0.95 0.95 0.93 0.94 0.95 0.93]
  warnings.warn(
C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.92       0.96       0.94666667 0.86666667 0.86666667 0.96
 0.95333333 0.96666667 0.85333333 0.85333333 0.96       0.96
 0.96666667 0.84666667 0.84666667 0.96666667 0.96       0.97333333
 0.86666667 0.86666667 0.96       0.98       0.96       0.85333333
 0.85333333 0.96666667 0.97333333 0.96666667 0.84666667 0.84666667
 0.966666

Fitting 10 folds for each of 69 candidates, totalling 690 fits


C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.88 0.94 0.92 0.8  0.8  0.92 0.94 0.96 0.78 0.78 0.94 0.94 0.95 0.77
 0.77 0.95 0.94 0.97 0.8  0.8  0.95 0.95 0.95 0.78 0.78 0.95 0.95 0.95
 0.77 0.77 0.96 0.94 0.95 0.8  0.8  0.94 0.95 0.95 0.78 0.78 0.95 0.95
 0.95 0.77 0.77 0.95 0.95 0.95 0.8  0.8  0.94 0.95 0.95 0.78 0.78 0.95
 0.95 0.95 0.77 0.77 0.94 0.95  nan 0.94 0.95 0.95 0.95 0.95 0.94]
  warnings.warn(
C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.92       0.96       0.94666667 0.86666667 0.86666667 0.94666667
 0.96       0.97333333 0.85333333 0.85333333 0.96       0.96
 0.96666667 0.84666667 0.84666667 0.96666667 0.96       0.98
 0.86666667 0.86666667 0.96666667 0.96666667 0.96666667 0.85333333
 0.85333333 0.96666667 0.96666667 0.96666667 0.84666667 0.84666667
 0.973333

KeyError: "['param_clf__s'] not found in axis"